# Making pairwise alignments with clustalo

In [1]:
import os
import glob
import Bio
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio import AlignIO

In [2]:
os.chdir("../data/families_seq/") # newfiles4 - version of working directory

##### Here we prepare our sample for Clustal

In [3]:
# Here you can check how much sequences contain your sample-file
dictt = SeqIO.to_dict(SeqIO.parse("sample.fasta", "fasta"))
fname_list = list(dictt.keys())
len(fname_list)

393

##### Now we are going to make a lot of PAIRWISE-ALIGNMENT files

In [ ]:
i = 0
# !mkdir clustalo
seq1, seq2 = fname_list[0], fname_list[2]


for num1 in range(i, len(fname_list)): #(i, len(fname_list)):     (len(fname_list))
    for num2 in range(i + 1, len(fname_list)): #(i + 1, len(fname_list)):    (len(fname_list))
        seq1, seq2 = fname_list[num1], fname_list[num2]
        
        out = open('../data/depo.fasta', 'w')
        out.write(">" + str(dictt[seq1].id) + '\n' + str(dictt[seq1].seq) + '\n' + ">" + str(dictt[seq2].id) + '\n' + str(dictt[seq2].seq) + '\n')
        out.close()
        ! clustalo -i ./depo.fasta -o ./'{seq1}_{seq2}'.fasta
    i += 1

In [ ]:
# To check how much pairwise-alignment files in your working directory
sum(list(range(62)))

In [ ]:
! rm depo.fasta

# Making matrices

##### Firstly, we'll check, if there're the number of files we need in our 'clustalo' directory

In [ ]:
len(fname_list)**2 == len(glob.glob('*.fasta'))

In [ ]:
#os.mkdir('./matrices')
os.chdir('./matrices/')
os.getcwd()

##### Making matrices

In [ ]:
'''
This function returns you one matrix with shape == (1, 1200, 10)
'''
def make_matrix(fname):
    # input file with pairwise alignment
    alignment = AlignIO.read(fname, "fasta")
    out_name = str(fname)[:-6]
    
    # making file for matrix
    out = open('mat_' + str(out_name) + '.txt', 'a')
    for record in alignment:
        out.write(str(record.seq) + '\n')
        alignment_len = len(str(record.seq).strip())
    out.close()    
    
    # making dataset 2 x alignment_len
    fname = open('mat_' + str(out_name) + '.txt', 'r')
    #print(fname)
    seq1 = list(fname.readline().strip())
    seq2 = list(fname.readline().strip())
    
    # print(len(seq1), len(seq2))
    
    
    fname.close()
    data = pd.DataFrame(np.array((seq1, seq2)))
    
    # making ZERO-matrix and dataset 10 x alignment_len
    matrix = np.zeros((10, 1200))  #((10, 1200)) # or ((10, 1200))
    mat_data = pd.DataFrame(matrix)
    
    # [A1 T1 G1 C1 GAP1][A2 T2 G2 C2 GAP2]
    # [00 01 02 03 0004][05 06 07 08 0009]
    # print(alignment_len)
    for num in range(alignment_len):
        sym1 = data[num][0]
        sym2 = data[num][1]
        
        if sym1: #== 'A' or sym1 == 'T' or sym1 == 'G' or sym1 == 'C' or sym1 == '-':
            if sym1 == 'A': mat_data[num][0] = 1
            elif sym1 == 'T': mat_data[num][1] = 1
            else:
                if sym1 == 'G': mat_data[num][2] = 1
                elif sym1 == 'C': mat_data[num][3] = 1
                else: mat_data[num][4] = 1
            
        if sym2: #== 'A' or sym2 == 'T' or sym2 == 'G' or sym2 == 'C' or sym2 == '-':
            if sym2 == 'A': mat_data[num][5] = 1
            elif sym2 == 'T': mat_data[num][6] = 1
            else:
                if sym2 == 'G': mat_data[num][7] = 1
                elif sym2 == 'C': mat_data[num][8] = 1
                else: mat_data[num][9] = 1    
    matrix1 = mat_data.astype(int)
    # print(matrix1)
    
    # Matrix reshaping
    matrix = np.swapaxes(matrix, 1, 0).reshape(1, 1200, 10)
    return matrix
    
    # Saving binary file in .npy format
     #print(matrix.shape)
    
    os.remove('mat_' + str(out_name) + '.txt')
    #np.save(str(out_name) + '.npy', matrix)

##### OLD VARIANT; SEE NEW VARIANT BELOW

In [ ]:
 MakeMatrix = make_matrix('./')

In [ ]:
'''
I check if our sequences have the same length, don't look at this cell
'''
for file in glob.glob("mat*.txt"):
    with open(file, 'r') as fname:
        seq1 = fname.readline().strip()
        seq2 = fname.readline().strip()
        if (len(seq1) == len(seq2)) != True:
            print(fname)

In [ ]:
# We shoul copy all files with pairwise alignments into the our working directory
mat = int(0)
sp = []
for file in glob.glob("*.fasta"):
    print(file)
    sp.append(file)
    if type(mat) == int:
        mat = make_matrix(file)
    else:
        nwm = make_matrix(file)
        mat = np.concatenate((mat, nwm), axis=0)
mat.shape
print(len(sp))
# mat_RF02272_JH835498.1,4_RF03115_URS0000D65EA6_682634,5.txt

In [ ]:
np.save('./ncRNApair_datav5.npy', mat)

##### Getting binary vector (2500,)

In [ ]:
pre_vec = []
for pair in sp:
   # print((int(pair.split(",")[1][0]), int(pair.split(",")[2][0])))
    check = (int(pair.split(",")[1][0]) == int(pair.split(",")[2][0]))
    if check == True:
        pre_vec.append(1)
    else:
        pre_vec.append(0)
bin_vec = np.array(pre_vec)

In [ ]:
np.save('./ncRNApair_labelv5.npy', bin_vec)

##### NEW VARIANT; with more adequate order (in accordance with famlables.txt)

In [ ]:
''' 
Here we make a list of ordered file-names 
We got *fam70_test.txt* earlier using Creating_sample.ipynb
'''

new_sp = []
fnames = []
order = []
with open("fam70_test.txt", 'r') as fname:
    for string in fname:
        new_sp.append(string.strip()[:-2])
for file in glob.glob("*.fasta"):
    fnames.append(file)
for i in new_sp:
    for j in fnames:
        if i in j.split(",")[0]:
            order.append(j)

In [ ]:
# Checking the length of our list
len(order)

In [ ]:
# We should copy all files with pairwise alignments into the our working directory
mat = int(0)

for file in order: # glob.glob("*.fasta"):
    if type(mat) == int:
        mat = make_matrix(file)
    else:
        mat = np.concatenate((mat, make_matrix(file)), axis=0)
mat.shape

In [ ]:
# Save your matrix into file to use it later for neural network (input)
np.save('./ncRNApair_datav7_test.npy', mat)

In [ ]:
# Getting a binary vector (are the sequences belong to the same family)
pre_vec = []
for pair in order:
    check = (int(pair.split(",")[1][0]) == int(pair.split(",")[2][0]))
    if check == True:
        pre_vec.append(0) # old version: pre_vec.append(1)
    else:
        pre_vec.append(1) # old version: pre_vec.append(0)
bin_vec = np.array(pre_vec)

In [ ]:
# Save your vector into file to use it later for neural network (input)
np.save('./ncRNApair_labelv7_test.npy', bin_vec)